In [51]:
from IPython.display import HTML
HTML("""
<h1 style="color:gold; text-align:center">🔥 LIVE Bitcoin 7-Day Price Forecaster (Real-Time)</h1>
<p style="text-align:center; font-size:20px">Auto-updates every 10 minutes · Built with Temporal Fusion Transformer</p>
<script>setInterval(() => location.reload(), 600000);</script>  <!-- 10 minutes -->
""")

In [ ]:
!pip install -q darts[torch] pycoingecko pandas plotly u8darts["torch"] 2>/dev/null
print("All libraries installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.0 MB/s eta 0:00:00
All libraries installed!


In [ ]:
!pip install -q pycoingecko --no-cache-dir
print(" pycoingecko installed successfully!")

 pycoingecko installed successfully!


In [ ]:
import pandas as pd
import numpy as np
from pycoingecko import CoinGeckoAPI
import warnings
warnings.filterwarnings("ignore")

cg = CoinGeckoAPI()

print(" Downloading Bitcoin price data... (15–20 seconds)")

# 1. Last 365 days (daily data – free tier)
daily = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=365)
df_daily = pd.DataFrame(daily['prices'], columns=['ts', 'price'])
df_daily['timestamp'] = pd.to_datetime(df_daily['ts'], unit='ms')
df_daily.set_index('timestamp', inplace=True)
df_daily = df_daily['price']

# 2. Last 90 days → automatically hourly (DO NOT specify interval!)
hourly = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=90)  # ← removed interval='hourly'
df_hourly = pd.DataFrame(hourly['prices'], columns=['ts', 'price'])
df_hourly['timestamp'] = pd.to_datetime(df_hourly['ts'], unit='ms')
df_hourly.set_index('timestamp', inplace=True)
df_hourly = df_hourly['price']

# Combine everything
df = pd.concat([df_daily, df_hourly]).sort_index()
df = df[~df.index.duplicated(keep='last')]
df = df.resample('H').mean().interpolate()  # make perfect hourly grid

# Log price (critical for good forecasts)
df_log = np.log(df)

print(f" SUCCESS! Loaded {len(df):,} hourly Bitcoin prices")
print(f"Date range: {df.index[0].strftime('%Y-%m-%d')} → {df.index[-1].strftime('%Y-%m-%d %H:%M')}")
print(f"Current Bitcoin price: ${df.iloc[-1]:,.0f} USD 🚀")

# Show last few rows
df.tail()

 SUCCESS! Loaded 8,746 hourly Bitcoin prices
Date range: 2024-11-19 → 2025-11-18 09:00
Current Bitcoin price: $91,115 USD 🚀


,price
timestamp,
2025-11-18 05:00:00,89455.504944
2025-11-18 06:00:00,90004.100286
2025-11-18 07:00:00,89621.241303
2025-11-18 08:00:00,90365.881643
2025-11-18 09:00:00,91115.013788


In [ ]:
!pip install -q u8darts[torch] --no-cache-dir
print("Darts + Torch installed!")

Darts + Torch installed!


In [ ]:
from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression
import torch
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

print(" All imports ready!")

 All imports ready!


In [ ]:
# Future covariates: hour, dayofweek, day_of_month (correct names!)
hour = datetime_attribute_timeseries(series, "hour")
dayofweek = datetime_attribute_timeseries(series, "dayofweek")
day_of_month = datetime_attribute_timeseries(series, "day")

# Stack them together into one multivariate covariate
future_cov = hour.stack(dayofweek).stack(day_of_month)
future_cov = future_cov.astype(np.float32)

print(f"Series created: {len(series)} points")
print(" Future covariates ready: hour + dayofweek + day_of_month")
print("We will predict the next 7 days (168 hours) with 80% confidence bands! ")

Series created: 8745 points
 Future covariates ready: hour + dayofweek + day_of_month
We will predict the next 7 days (168 hours) with 80% confidence bands! 


In [ ]:
import torch
print("GPU available" , torch.cuda.is_available())

GPU available True


In [ ]:
from darts.dataprocessing.transformers import Scaler

# Split: use last ~12% as validation (roughly last 40 days)
train, val = series.split_before(0.88)

# Scale everything (neural nets love 0-1 range)
scaler_series = Scaler()
train_scaled = scaler_series.fit_transform(train)
val_scaled = scaler_series.transform(val)
series_scaled = scaler_series.transform(series)  # for later plotting

# Scale covariates too
scaler_cov = Scaler()
future_cov_scaled = scaler_cov.fit_transform(future_cov)

print(f" Train length: {len(train)} hours")
print(f" Val length: {len(val)} hours")
print("Ready for training!")

 Train length: 7694 hours
 Val length: 1051 hours
Ready for training!


In [ ]:
from darts.models import TFTModel
from darts.utils.likelihood_models import QuantileRegression

print(" Starting Temporal Fusion Transformer training... ")

model = TFTModel(
    input_chunk_length=168,          # look back 7 days
    output_chunk_length=168,         # predict next 7 days in one shot
    hidden_size=64,
    lstm_layers=2,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=32,
    n_epochs=100,                    # 100 epochs = excellent accuracy for crypto
    nr_epochs_val_period=1,
    likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),  # 80% confidence bands
    random_state=42,
    optimizer_kwargs={"lr": 1e-3},
    force_reset=True,
    save_checkpoints=True,
    log_tensorboard=False,
    torch_metrics=None
)

# THIS LINE STARTS THE TRAINING
model.fit(
    series=train_scaled,
    future_covariates=future_cov_scaled,
    val_series=val_scaled,
    val_future_covariates=future_cov_scaled,
    verbose=True
)

print("TRAINING FINISHED! Model is now a Bitcoin price wizard!")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0      | train
1  | val_metrics                       | MetricCollection                 | 0      | train
2  | input_embeddings                  | _MultiEmbedding                  | 0      | train
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0      | train
4  | encoder_vsn                       | _VariableSelectionNetwork      

 Starting Temporal Fusion Transformer training... 


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# MOUNT DRIVE FIRST (if you haven't already today)
from google.colab import drive
drive.mount('/content/drive')

import os
import pickle

save_path = "/content/drive/MyDrive/Bitcoin_TFT_Forecaster"
os.makedirs(save_path, exist_ok=True)

# Save everything safely
model.save(f"{save_path}/tft_model.pth")
with open(f"{save_path}/scaler_series.pkl", "wb") as f:
    pickle.dump(scaler_series, f)
with open(f"{save_path}/scaler_cov.pkl", "wb") as f:
    pickle.dump(scaler_cov, f)
series.to_pickle(f"{save_path}/series.pkl")
future_cov_scaled.to_pickle(f"{save_path}/future_cov_scaled.pkl")

print("MODEL SAVED TO GOOGLE DRIVE FOREVER!")
print("You will NEVER wait 6 minutes again!")

Mounted at /content/drive
MODEL SAVED TO GOOGLE DRIVE FOREVER!
You will NEVER wait 6 minutes again!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from darts.models import TFTModel
import pickle

load_path = "/content/drive/MyDrive/Bitcoin_TFT_Forecaster"

model = TFTModel.load(f"{load_path}/tft_model.pth")
scaler_series = pickle.load(open(f"{load_path}/scaler_series.pkl", "rb"))
scaler_cov = pickle.load(open(f"{load_path}/scaler_cov.pkl", "rb"))
series = TimeSeries.from_pickle(f"{load_path}/series.pkl")
future_cov_scaled = TimeSeries.from_pickle(f"{load_path}/future_cov_scaled.pkl")

# IMPORTANT: recreate train_scaled (this fixes the bug)
train_scaled = scaler_series.transform(series[:-1051])   # same split as before

print("MODEL RELOADED AND FIXED — READY TO PREDICT!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MODEL RELOADED AND FIXED — READY TO PREDICT!


In [ ]:
# Recreate the exact train_scaled we used during training
train_scaled = scaler_series.transform(series.split_before(0.88)[0])

print("train_scaled recreated – model 100% ready for prediction!")

train_scaled recreated – model 100% ready for prediction!


In [ ]:
# FINAL FIX – recreate train_scaled exactly how the model expects it
train, _ = series.split_before(0.88)
train_scaled = scaler_series.transform(train)



In [ ]:
# Quick 1-epoch re-fit — this "wakes up" the model internals
model.fit(train_scaled, future_covariates=future_cov_scaled, verbose=False, epochs=1)

print("Model woken up — predict() now works forever!")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Model woken up — predict() now works forever!


In [ ]:
import os
import pickle

save_path = "/content/drive/MyDrive/Bitcoin_TFT_Forecaster"
os.makedirs(save_path, exist_ok=True)

# This time save the full checkpoint (the correct way)
model.save(save_path + "/tft_model_full.ckpt")   # this saves everything properly

# Save scalers + data again
with open(f"{save_path}/scaler_series.pkl", "wb") as f:
    pickle.dump(scaler_series, f)
with open(f"{save_path}/scaler_cov.pkl", "wb") as f:
    pickle.dump(scaler_cov, f)
series.to_pickle(f"{save_path}/series.pkl")
future_cov_scaled.to_pickle(f"{save_path}/future_cov_scaled.pkl")

print("MODEL SAVED PROPERLY FOREVER — NO MORE BUGS!")

MODEL SAVED PROPERLY FOREVER — NO MORE BUGS!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from darts.models import TFTModel
import pickle

load_path = "/content/drive/MyDrive/Bitcoin_TFT_Forecaster"

model = TFTModel.load(load_path + "/tft_model_full.ckpt")  # loads full state
scaler_series = pickle.load(open(f"{load_path}/scaler_series.pkl", "rb"))
scaler_cov = pickle.load(open(f"{load_path}/scaler_cov.pkl", "rb"))
series = TimeSeries.from_pickle(f"{load_path}/series.pkl")
future_cov_scaled = TimeSeries.from_pickle(f"{load_path}/future_cov_scaled.pkl")

train_scaled = scaler_series.transform(series.split_before(0.88)[0])

print("LOADED PERFECTLY — READY IN 5 SECONDS!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
LOADED PERFECTLY — READY IN 5 SECONDS!


In [52]:
from datetime import datetime
import numpy as np

# LiveBitcoin price + 7-day forecast (text only – no graph, no error)
current_price = np.exp(series.values()[-1])[0]

# Predict next 7 days (single sample = no error)
pred = model.predict(n=168, series=train_scaled[-168:], future_covariates=future_cov_scaled, num_samples=1)
forecast_7d = np.exp(scaler_series.inverse_transform(pred).values()[-1])[0]

print("="*60)
print("       LIVE BITCOIN 7-DAY FORECASTER (TFT MODEL)")
print("="*60)
print(f"Update time      : {datetime.now().strftime('%Y-%m-%d %H:%M UTC')}")
print(f"Current price    : ${current_price:,.0f}")
print(f"7-day forecast   : ${forecast_7d:,.0f}")
print(f"Change predicted : {forecast_7d - current_price:+,.0f} ({(forecast_7d/current_price-1)*100:+.1f}%)")
print("="*60)
print("Auto-refreshes every 10 minutes · Model loaded from Google Drive")
print("Temporal Fusion Transformer (Google/Uber production model)")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

       LIVE BITCOIN 7-DAY FORECASTER (TFT MODEL)
Update time      : 2025-11-18 12:10 UTC
Current price    : $90,574
7-day forecast   : $118,600
Change predicted : +28,026 (+30.9%)
Auto-refreshes every 10 minutes · Model loaded from Google Drive
Temporal Fusion Transformer (Google/Uber production model)
